# Results Analysis
## 1. Setup
### 1.1 Imports

In [1]:
from helpers import *
from ipywidgets import HTML
from scipy.stats.mstats import hmean
import pandas as pd

pd.set_option('styler.format.precision', 4)

### 1.2 Constants
These are used to filter and rename the columns and metrics.

In [2]:
MARKS = [ 'CodeXGLUE', 'D2A', 'Draper VDISC' ]
COMBO = [ 'All', 'All Balanced', 'CodeXGLUE+D2A' ]
# MARKS = [ 'codexglue', 'd2a', 'draper' ]
# COMBO = [ 'all', 'allbalanced', 'codexglued2a' ]
METRICS = [ 'avg_prec', 'accuracy', 'f1', 'precision', 'recall', 'cet_score' ]
RENAMES = [
    ('codexglue', 'CodeXGLUE'), ('d2a', 'D2A'), ('draper', 'Draper VDISC'),
    ('all', 'All'), ('allbalanced', 'All Balanced'), ('codexglued2a', 'CodeXGLUE+D2A'),
    ('codebert', 'CodeBERT'), ('cotext', 'CoTexT'), ('linevul', 'LineVul'),
    ('regvd', 'ReGVD'), ('textcnn', 'TextCNN'), ('xgboost', 'XGBoost')
]
METRIC_NAMES = { 'accuracy': 'Acc.', 'avg_prec': 'AP', 'f1': 'F1', 'precision': 'P', 'recall': 'R' }

### 1.3 Datasets
Load the code execution task results and predictions.

In [3]:
predictions = load_predictions()
add_truth(predictions, 'd2a', '../data/csv/d2a/valid.csv')
add_truth(predictions, 'codexglue', '../data/csv/codexglue/test.csv')
add_truth(predictions, 'draper', '../data/csv/draper/test.csv')
tasks = pd.read_csv('tasks.csv')

## 2. Research Questions
### 2.1 RQ1 - How well do models learn code execution tasks?

We will create a code execution task (CET) score as the harmonic mean and display the results.

In [4]:
tasks['cet_score'] = tasks.iloc[:, 1:].apply(hmean, axis=1)
tasks.style.background_gradient('RdYlGn', axis=None)

,model,task1,task2,task3,task4,task5,task6,cet_score
0,codebert,0.9994,0.9994,0.9788,0.9835,0.9979,0.4990,0.8516
1,cotext,0.9998,0.9998,0.9479,0.9909,0.9992,0.5010,0.8497
2,linevul,0.9994,0.9993,0.9724,0.9223,0.9985,0.4990,0.8428
3,regvd,0.9462,0.9263,0.4989,0.5002,0.6110,0.4990,0.6135
4,textcnn,0.9809,0.4999,0.4989,0.4998,0.4999,0.5010,0.5444
5,xgboost,0.8869,0.7405,0.5241,0.5135,0.5639,0.5011,0.5939


All of the models are able partially learn at least one task. Unexpectedly, ReGVD fails to learn quite a few. XGBoost never fully learns a task, but ends up with a higher mean than TextCNN.

### 2.2 RQ2 - Is SVD performance correlated with code execution ability?

Calculate the metrics and display the first few rows.

In [5]:
metrics = load_metrics(predictions)
metrics = pd.merge(metrics, tasks[['model', 'cet_score']], on='model')
for search, replace in RENAMES: metrics = metrics.replace(search, replace)

metrics.groupby(['model']).count()[['test']]

Output()

,test
model,
CoTexT,18
CodeBERT,18
LineVul,18
ReGVD,18
TextCNN,18
XGBoost,18


#### 2.2.1 Correlation by Metric
Let's look at the correlation of each metric with the CET score.

In [6]:
def cet2metric(ds):
    both = metrics.loc[metrics.train.isin(ds), METRICS].corr()[['cet_score']]
    same = metrics.loc[metrics.train.isin(ds) & (metrics.train == metrics.test), METRICS].corr()[['cet_score']]
    diff = metrics.loc[metrics.train.isin(ds) & (metrics.train != metrics.test), METRICS].corr()[['cet_score']]

    corr = pd.concat([both, same, diff], axis=1)
    corr.columns = [ 'All', 'Train = Test', 'Train ≠ Test' ]
    corr = corr.rename(index=METRIC_NAMES)
    return corr.sort_values(by='All', ascending=False).iloc[1:, :]


cet2metric(MARKS).style\
    .background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1)\
    .format(precision=2)

,All,Train = Test,Train ≠ Test
P,0.16,0.28,0.15
F1,0.15,0.53,0.01
Acc.,0.08,0.14,0.07
AP,0.05,0.23,0.00
R,0.03,0.30,-0.07


AP correlation is strong for training/testing on same dataset, but extremely weak when train/test on different datasets is included.

#### 2.2.2 AP Correlation by Training and Test Dataset

In [7]:
def cet2ap(ds):
    corr = metrics.loc[metrics.train.isin(ds), ['train', 'test', 'avg_prec', 'cet_score']]
    corr = corr.groupby(['train', 'test']).corr()[['cet_score']].reset_index()#.rename(columns={'level_2': 'corr'})
    corr = corr[corr.level_2 == 'avg_prec']
    corr = corr.pivot(index='train', columns='test', values='cet_score')
    corr.columns.name = 'Test'
    corr.index.name = 'Train'
    return corr

cet2ap(MARKS).style\
             .background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1)\
             .format(precision=2)

Test,CodeXGLUE,D2A,Draper VDISC
Train,,,
CodeXGLUE,0.20,-0.10,0.28
D2A,-0.65,0.15,0.14
Draper VDISC,0.36,0.47,0.65


This is made clear when considering AP correlation to CET score based on the training and test dataset. Draper appears to be driving the positive results while D2A has a strong negative correlation.

#### 2.2.3 Metrics Correlation by Training and Test Datasets

In [24]:
def cet2data(ds):
    group = [ 'train', 'test' ]
    corr = metrics.loc[metrics.train.isin(ds), group + METRICS]
    corr = corr.groupby(group).corr()[['cet_score']].reset_index().rename(columns={'level_2': 'corr'})
    corr = corr.pivot(index=group, columns='corr', values='cet_score').drop(columns='cet_score')
    
    # Get total correlation
    corr2 = metrics.loc[metrics.train.isin(ds), group + METRICS]
    corr2 = corr2.groupby('train').corr(numeric_only=True)[['cet_score']].reset_index().rename(columns={'level_1': 'corr'})
    corr2 = corr2.pivot(index='train', columns='corr', values='cet_score').drop(columns='cet_score')
    corr2 = corr2.reset_index()
    corr2['test'] = 'Total'
    corr2 = corr2.set_index(group)
    
    corr = pd.concat([corr, corr2], axis=0).sort_values(by=['train', 'test'])
    corr.columns.name = None
    corr.index.names = [ 'Train', 'Test' ]
    corr = corr.rename(columns=METRIC_NAMES)
    return corr

marks = cet2metric(MARKS).transpose()
marks.index.name = 'Train'
marks = marks.reset_index()
marks['Test'] = ''
marks = marks.set_index(['Train', 'Test'])

combo = cet2metric(COMBO).transpose()
combo.index.name = 'Train'
combo = combo.reset_index()
combo['Train'] = ['Combo All', 'Combo Train = Test', 'Combo Train != Test']
combo['Test'] = ''
combo = combo.set_index(['Train', 'Test'])

corr = cet2data(MARKS + COMBO)
corr = pd.concat([corr, marks, combo], axis=0)

corr.style\
    .background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1)\
    .format(precision=2)

#### 2.2.4 AP Drop On Train/Test Difference

How much worse do models get when they aren't trained and tested on the same dataset?

In [9]:
group = [ 'train' ]
same = metrics.loc[metrics.train.isin(MARKS) & (metrics.train == metrics.test), group + METRICS]
diff = metrics.loc[metrics.train.isin(MARKS) & (metrics.train != metrics.test), group + METRICS]
same = same.groupby(group).mean()
diff = diff.groupby(group).mean()
diff = (diff - same)
diff.index.name = None

diff.rename(columns=METRIC_NAMES)\
    .drop(columns='cet_score')\
    .style.background_gradient(cmap='bwr_r', vmin=-1, vmax=1)\
    .format(precision=2)

,AP,Acc.,F1,P,R
CodeXGLUE,-0.31,-0.16,-0.12,-0.29,0.29
D2A,-0.39,-0.09,-0.35,-0.38,-0.17
Draper VDISC,0.09,-0.40,-0.24,-0.01,-0.44


### 2.3 RQ3 - Does combining datasets improve SVD performance?

#### 2.3.1 AP Difference by Train and Test Dataset

Let's look at how the AP changes for each benchmark.

In [10]:
combo = metrics[metrics.train.isin(COMBO)].groupby(['model', 'train', 'test'])[['avg_prec']].first()
marks = metrics[metrics.train.isin(MARKS)].sort_values(by='avg_prec', ascending=False).groupby(['model', 'test'])[['avg_prec']].first()
diff = (combo - marks).reset_index().pivot(index=['model', 'train'], columns='test', values='avg_prec')

diff.style\
    .background_gradient(cmap='bwr_r', vmin=-1, vmax=1)\
    .format(precision=2)

In [11]:
metrics.loc[metrics.model == 'CoTexT']

,model,train,test,has_logits,avg_prec,accuracy,f1,precision,recall,cet_score
18,CoTexT,All,D2A,False,0.5595,0.5822,0.6448,0.5751,0.7338,0.849695
19,CoTexT,All,Draper VDISC,False,0.3632,0.9453,0.5797,0.5774,0.5821,0.849695
20,CoTexT,All,CodeXGLUE,False,0.5283,0.6120,0.5853,0.5749,0.5960,0.849695
21,CoTexT,All Balanced,CodeXGLUE,False,0.4762,0.5007,0.6300,0.4776,0.9251,0.849695
22,CoTexT,All Balanced,Draper VDISC,False,0.2733,0.8887,0.4691,0.3394,0.7594,0.849695
23,CoTexT,All Balanced,D2A,False,0.5153,0.5134,0.6771,0.5153,0.9870,0.849695
24,CoTexT,CodeXGLUE,D2A,False,0.5179,0.5067,0.5702,0.5186,0.6331,0.849695
25,CoTexT,CodeXGLUE,Draper VDISC,False,0.0684,0.5475,0.1261,0.0720,0.5038,0.849695
26,CoTexT,CodeXGLUE,CodeXGLUE,False,0.5329,0.6197,0.5740,0.5912,0.5578,0.849695
27,CoTexT,CodeXGLUE+D2A,CodeXGLUE,False,0.5455,0.6358,0.5786,0.6175,0.5442,0.849695


#### 2.3.2 Mean AP Difference by Training and Test Datasets

Let's look at the mean AP difference by training dataset to compress the information above.

In [12]:
diff.reset_index().groupby('train').mean(numeric_only=True)\
    .style.background_gradient(cmap='bwr_r', vmin=-1, vmax=1)\
    .format(precision=2)

test,CodeXGLUE,D2A,Draper VDISC
train,,,
All,-0.07,-0.10,0.00
All Balanced,-0.10,-0.15,-0.11
CodeXGLUE+D2A,0.00,-0.05,-0.33


#### 2.3.3 Metric Differences by Train/Test Pair

In [25]:
ap = metrics.copy()
ap['name'] = ''
ap.loc[ap.train.isin(COMBO), 'name'] = 'On Merged'
ap.loc[ap.train == ap.test, 'name'] = '= Test'
ap.loc[ap.train.isin(MARKS) & (ap.train != ap.test), 'name'] = '≠ Test'
# ap['type'] = 'mark'
# ap.loc[ap.train.isin(COMBO), 'type'] = 'combo'
# ap['same'] = False
# ap.loc[ap.train == ap.test, 'same'] = True

ap = ap.groupby('name').mean(numeric_only=True)[['avg_prec', 'accuracy', 'f1', 'precision', 'recall']]
ap = ap.reset_index()
ap = ap.melt('name').pivot(index='name', columns='variable', values='value')
ap = ap.rename(columns=METRIC_NAMES)
ap.columns.name = None
ap.index.name = 'Train'
ap = ap.reset_index()
# ap.columns = [ 'On Merged', '!= Test', '== Test']
# ap.index.name = None

rint(ap.style.background_gradient(cmap='Blues')\
  .format(precision=2)

,Train,Acc.,AP,F1,P,R
0,= Test,0.71,0.56,0.49,0.55,0.51
1,On Merged,0.58,0.46,0.49,0.44,0.71
2,≠ Test,0.49,0.35,0.25,0.32,0.40


### 2.4 RQ4 - Does combining datasets improve SVD/CET correlation?

#### 2.4.1 Correlation by Metric

In [14]:
marks = cet2metric(MARKS)
combo = cet2metric(COMBO)

display(HTML('<h4 style="margin:0">Correlation with Benchmarks</h4>'))
display(marks.style.background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1).format(precision=2))
display(HTML('<h4 style="margin:1rem 0 0">Correlation with Merged</h4>'))
display(combo.style.background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1).format(precision=2))
display(HTML('<h4 style="margin:1rem 0 0">Merged - Benchmarks</h4>'))
display((combo - marks).style.background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1).format(precision=2))

HTML(value='<h4 style="margin:0">Correlation with Benchmarks</h4>')

,All,Train = Test,Train ≠ Test
P,0.16,0.28,0.15
F1,0.15,0.53,0.01
Acc.,0.08,0.14,0.07
AP,0.05,0.23,0.00
R,0.03,0.30,-0.07


HTML(value='<h4 style="margin:1rem 0 0">Correlation with Merged</h4>')

,All,Train = Test,Train ≠ Test
F1,0.23,nan,0.23
Acc.,0.22,nan,0.22
AP,0.18,nan,0.18
P,0.12,nan,0.12
R,0.11,nan,0.11


HTML(value='<h4 style="margin:1rem 0 0">Merged - Benchmarks</h4>')

,All,Train = Test,Train ≠ Test
AP,0.13,nan,0.18
Acc.,0.14,nan,0.15
F1,0.08,nan,0.22
P,-0.04,nan,-0.03
R,0.08,nan,0.18


In [15]:
t = pd.concat([marks, combo[['All']]], axis=1)
t.columns.values[-1] = 'Merged'
# t.columns = [ 'Benchmarks', 'Combined' ]
# t = t.sort_values('Combined', ascending=False)
t = t.transpose()
t = t[sorted(t)]
t.sort_values(by='AP', ascending=False).style.background_gradient(cmap='Blues', axis=None).format(precision=2)

,AP,Acc.,F1,P,R
Train = Test,0.23,0.14,0.53,0.28,0.30
Merged,0.18,0.22,0.23,0.12,0.11
All,0.05,0.08,0.15,0.16,0.03
Train ≠ Test,0.00,0.07,0.01,0.15,-0.07


#### 2.4.2 Correlation by Training and Test Dataset

In [28]:
apm = cet2ap(MARKS)
apm['mean'] = apm.mean(axis=1)
apm = apm.sort_values(by='mean', ascending=False)

apc = cet2ap(COMBO)
apc['mean'] = apc.mean(axis=1)
apc = apc.sort_values(by='mean', ascending=False)
ap = pd.concat([apm, apc])
ap['mean'] = ap.mean(axis=1)
ap = ap.sort_values(by='mean', ascending=False)

ap.style\
  .background_gradient(cmap='bwr_r', axis=None, vmin=-1, vmax=1)\
  .format(precision=2)

Test,CodeXGLUE,D2A,Draper VDISC,mean
Train,,,,
All,0.78,0.82,0.70,0.77
All Balanced,0.53,0.87,0.40,0.60
Draper VDISC,0.36,0.47,0.65,0.49
CodeXGLUE,0.20,-0.10,0.28,0.13
CodeXGLUE+D2A,0.33,0.19,-0.18,0.11
D2A,-0.65,0.15,0.14,-0.12


## 3. Miscellaneous
### 3.1 Average Precision

In [17]:
ap = metrics.groupby(['model', 'train', 'test']).mean()[['avg_prec']].reset_index()
ap = ap.pivot(['model', 'train'], 'test', 'avg_prec')
ap['mean'] = ap.mean(axis=1)
ap.style.background_gradient(cmap='Blues')

/tmp/ipykernel_348876/3563847752.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  ap = ap.pivot(['model', 'train'], 'test', 'avg_prec')


In [18]:
# for metric in METRICS:
#     combo = metrics[metrics.train.isin(COMBO)].groupby(['model', 'train', 'test'])[[metric]].mean()
#     marks = metrics[metrics.train.isin(MARKS)].sort_values(by='avg_prec', ascending=False).groupby(['model', 'test'])[[metric]].first()
#     diff = (combo - marks).reset_index().pivot(['model', 'train'], 'test', metric)
#     # display(combo)
#     # display(marks)
#     display(HTML(f'<h4>{metric}</h4>'))
#     display(diff.reset_index().groupby('train').mean().style.background_gradient(cmap='bwr_r', vmin=-1, vmax=1))
#     display(diff.style.background_gradient(cmap='bwr_r', vmin=-1, vmax=1))

In [19]:
top = metrics.sort_values(by='avg_prec', ascending=False)
# top = top.loc[top.train.isin(COMBO)]
top = top.groupby(['test', 'model']).first()
top = top.reset_index().sort_values(by=['test', 'avg_prec'], ascending=[True, False])
top.style.background_gradient(cmap='Blues')

,test,model,train,has_logits,avg_prec,accuracy,f1,precision,recall,cet_score
2,CodeXGLUE,LineVul,CodeXGLUE,True,0.7154,0.6526,0.5359,0.6937,0.4367,0.8428
4,CodeXGLUE,TextCNN,CodeXGLUE,True,0.6975,0.6501,0.4963,0.7325,0.3753,0.5444
1,CodeXGLUE,CodeBERT,CodeXGLUE,True,0.6821,0.6373,0.5310,0.6538,0.4470,0.8516
5,CodeXGLUE,XGBoost,CodeXGLUE,True,0.5945,0.5838,0.4669,0.5672,0.3968,0.5939
3,CodeXGLUE,ReGVD,CodeXGLUE+D2A,True,0.5551,0.5794,0.5499,0.5408,0.5594,0.6135
0,CodeXGLUE,CoTexT,CodeXGLUE+D2A,False,0.5455,0.6358,0.5786,0.6175,0.5442,0.8497
7,D2A,CodeBERT,D2A,True,0.7033,0.6208,0.6523,0.6199,0.6883,0.8516
8,D2A,LineVul,D2A,True,0.6961,0.6124,0.6078,0.6370,0.5812,0.8428
10,D2A,TextCNN,D2A,True,0.6602,0.5638,0.3401,0.7791,0.2175,0.5444
11,D2A,XGBoost,D2A,True,0.6602,0.5956,0.6440,0.5908,0.7078,0.5939


In [20]:
table = metrics.melt(['model', 'train', 'test'])
table = table.loc[
    ((table.variable == 'accuracy') & table.test.isin(['CodeXGLUE', 'D2A'])) |
    ((table.variable == 'f1') & (table.test == 'Draper VDISC')) |
    (table.variable == 'avg_prec')
]
table = table.sort_values(by='model')
table = table.pivot(index=['model', 'train'], columns=['test', 'variable'], values='value')

# Sort columns, always having AP 2nd.
cols = sorted(table)
cols[-1], cols[-2] = cols[-2], cols[-1]
table = table[cols].rename(columns=METRIC_NAMES)

table.columns.names = [ None, None ]
table.index.names = [ 'Model', 'Train' ]
table.style.highlight_max()

In [21]:
for name, group in table.groupby('Model'):
    display(HTML(f'<h4>{name}</h4>'))
    display(group.style.highlight_max())

HTML(value='<h4>CoTexT</h4>')

HTML(value='<h4>CodeBERT</h4>')

HTML(value='<h4>LineVul</h4>')

HTML(value='<h4>ReGVD</h4>')

HTML(value='<h4>TextCNN</h4>')

HTML(value='<h4>XGBoost</h4>')